# 영업소 현재차로 선행차량 정보추출

# Set Up

## Import

In [1]:
import pandas as pd
import numpy as np
import os

import warnings
from tqdm import tqdm

In [2]:
save_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/현재차로_선행차량정보'

## Dataset

### LC
* 차로변경 차량궤적 데이터셋

In [3]:
LC_dir = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/38_22-10차 회의 준비/221022_인접차량정보/차로변경차량정보'

In [4]:
LC_file_list = os.listdir(LC_dir)

In [5]:
LC_file_list_Seoul = LC_file_list[:6]
LC_file_list_Yangyang = LC_file_list[6:]

### Full
* 전체 차량궤적 데이터셋

In [6]:
data_dir_Yangyang = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/27_남양주영업소_드론영상분석/양양방향/01_density_by_target_LOS_concated_lanechange'

In [7]:
data_dir_Seoul = 'D:/OneDrive - 연세대학교 (Yonsei University)/Projects/Yonsei_TELab/003_도로상충_210517-/2차년도_2022/27_남양주영업소_드론영상분석/서울방향/01_density_by_target_LOS_concated_lanechange'

In [8]:
Full_file_list_Yangyang = os.listdir(data_dir_Yangyang)
Full_file_list_Yangyang = Full_file_list_Yangyang[-2:]
Full_file_list_Yangyang

['04_1_A.csv', '04_2_A.csv']

In [9]:
Full_file_list_Seoul = os.listdir(data_dir_Seoul)
Full_file_list_Seoul = Full_file_list_Seoul[:6]
Full_file_list_Seoul

['01_1_A.csv',
 '01_2_A.csv',
 '01_3_A.csv',
 '02_1_A.csv',
 '02_2_A.csv',
 '02_3_A.csv']

# Workflow
* LC 데이터셋을 바탕으로 마련한다.
* LC 데이터셋 같은 프레임 -> 현재차로 -> Local X가 가장 가까운, 가장 가까운차량(`LV1`)을 탐색 -> 해당 차량의 정보를 LC 데이터셋에 추가
* LC 데이터셋에 새로운 컬럼을 추가하기:
    * `LV1_ID` : 현재차로차량의 Vehicle ID
    * `LV1_Vehicle Speed` : 현재차로 차량의 속도
    * `LV1_x` : 현재차로 선행 차량의 Local X
    * `LV1_y` : 현재차로 선행 차량의 Local Y
* 다음의 파생변수를 생성함:
    * $V_{LV1}$ : 현재 차로의 선행차량($LV1$) 속도
    * $D_{LV1}$ : 현재 차로에서 대상 차량($HV$)과 선행 차량($LV1$) 사이의 초기 차간거리
    * $SD_{LV1}$ : $t_{LC}$ 동안(차로변경 시) 대상 차량($HV$)과 선행 차량($LV1$) 간의 최소안전거리
* 참고 : 
    * `idx = df[(df['Vehicle ID'] == veh) & (df['Frame ID'] == frm)]['Lane_change'].index[0]` # 차로변경된 행의 인덱스
    * `df.at[idx, 'Lane Identification Past'] = str(Lane_past)`

## 양양방향

## 서울방향

In [10]:
OBS_point_02_1 = {'U3' : (84.063, 103.113), 'U4' : (92.403, 79.683), 'U5' : (60.149, 74.817), 'U6' : (33.748, 69.464)}
OBS_point_02_2 = {'U1' : (60.149, 74.817), 'U2' : (33.748, 69.464)}

In [11]:
for LC_file, full_file in zip(LC_file_list_Seoul, Full_file_list_Seoul):
    
    LC_file_num = LC_file[-8:-4] # 파일번호
    
    LC_file_path = os.path.join(LC_dir, LC_file)
    LC_df = pd.read_csv(LC_file_path) # 각 LC 차량을 불러오기
    
    full_data_path = os.path.join(data_dir_Seoul, full_file)
    full_df = pd.read_csv(full_data_path)
    
    LC_df['LV1_ID'] = None # 현재차로 선행차량 Vehicle ID
    LC_df['LV1_Velocity'] = None # 현재차로 선행차량 Vehicle Velocity
    LC_df['LV1_x'] = None # 현재차로 선행차량 x좌표
    LC_df['LV1_y'] = None # 현재차로 선행차량 y좌표
    
    for i in tqdm(range(len(LC_df))): ## i = 0~ 인덱스
        row = LC_df.iloc[i] # LC_df의 각 행별로
        
        HV_x = row['Local X (m)'] ### 차량의 X좌표v
        HV_y = row['Local Y(m)'] ### 차량의 Y좌표
        HV_Lane_present = row['Lane_present'] ### 차량의 현재 차로
        frame = row['Frame ID']
        
        ### Full DF로부터 해당 프레임, 해당 present Lane에 존재하며, Local X보다 앞에 있는(=x값이 "작은") row 리스트를 뽑기
        target_df = full_df[(full_df['Frame ID'] == frame) & (full_df['Lane Identification'] == HV_Lane_present) & (full_df['Local X (m)'] < HV_x)]
        
        ### 선행 차량이 있을 경우, 선행 차량이 없을 경우를 나누기
        if len(target_df) >= 1: # 해당 프레임 선행 차량이 1대 이상 존재할 경우 : Local X값이 가장 "큰" 것의 값을 취하시오
            target_df = target_df[target_df['Local X (m)'] == target_df['Local X (m)'].max()]
            
            LC_df.at[i, 'LV1_ID'] = target_df['Vehicle ID'].iloc[0]
            LC_df.at[i, 'LV1_Velocity'] = target_df['Vehicle Velocity(km/h)'].iloc[0]
            LC_df.at[i, 'LV1_x'] = target_df['Local X (m)'].iloc[0]
            LC_df.at[i, 'LV1_y'] = target_df['Local Y(m)'].iloc[0]
            
        else: # 해당 프레임에서 선행 차량이 없을 경우 : obs_remain을 적용해야 함. PE방호벽에 따라서 다름.
            if LC_file_num == '04_1': # PE방호벽 영향을 받는 파일이며 현재 차로가 PE방호벽이 있는 차로인 경우
                OBS_lane_list = list(OBS_point_04_1.keys()) ### PE방호벽 영향을 받는 차로 리스트
                    
                if HV_Lane_present in OBS_lane_list: # PE방호벽 영향을 받는 차로이면
                    LC_df.at[i, 'LV1_ID'] = 'OBS' # PE방호벽밖에 없음을 의미함
                    LC_df.at[i, 'LV1_Velocity'] = 0 # PE방호벽의 속도는 0임
                    LC_df.at[i, 'LV1_x'] = OBS_point_04_1[HV_Lane_present][0] ### PE방호벽 x좌표
                    LC_df.at[i, 'LV1_y'] = OBS_point_04_1[HV_Lane_present][1] ### PE방호벽 y좌표
                    
            
                
            else: #선행차량이 없고 PE방호벽 영향을 안받는 파일인경우
                pass
        
    save_path = os.path.join(save_dir, LC_file)
    LC_df.to_csv(save_path)

100%|███████████████████████████████████████████████████████████████████████████████| 513/513 [00:01<00:00, 319.56it/s]
